In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
from keras import Sequential
from keras.layers import Conv2D,Dense,MaxPooling2D,BatchNormalization,Flatten,Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
base_dir, _ = os.path.splitext("../../../../Downloads/chest_xray")

In [3]:
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

train_normal = os.path.join(train_dir, 'NORMAL')
print ('Total training normal images:', len(os.listdir(train_normal)))
train_pneu = os.path.join(train_dir, 'PNEUMONIA')
print ('Total training pneu images:', len(os.listdir(train_pneu)))

val_normal = os.path.join(validation_dir, 'NORMAL')
print ('Total validation normal images:', len(os.listdir(val_normal)))
val_pneu = os.path.join(validation_dir, 'PNEUMONIA')
print ('Total validation pneu images:', len(os.listdir(val_pneu)))

test_dir = os.path.join(base_dir,'test')
test_normal = os.path.join(test_dir, 'NORMAL')
print ('Total test normal images:', len(os.listdir(test_normal)))
test_pneu = os.path.join(test_dir, 'PNEUMONIA')
print ('Total test pneu images:', len(os.listdir(test_pneu)))

Total training normal images: 1341
Total training pneu images: 3875
Total validation normal images: 8
Total validation pneu images: 8
Total test normal images: 234
Total test pneu images: 390


In [4]:
image_size = 256 # All images will be resized to 160x160
batch_size = 8

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
              %  rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
test_generator = test_datagen.flow_from_directory(
                test_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/opt/anaconda3/lib/python3.7/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [6]:
print("Number of layers in the base model: ", len(base_model.layers))

Number of layers in the base model:  155


In [7]:
base_model.trainable = True

In [8]:
# Fine tune from this layer onwards
fine_tune_at = 120

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [9]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dropout(.3),
  keras.layers.Dense(1, activation='sigmoid')
    
])

In [10]:
# model.add()

In [11]:
model.compile(optimizer = Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# epochs = 15
# steps_per_epoch = train_generator.n // batch_size
# validation_steps = validation_generator.n // batch_size

# history = model.fit(train_generator,
#                               steps_per_epoch = steps_per_epoch,
#                               epochs=epochs,
#                               workers=4,
#                               validation_data=validation_generator,
#                               validation_steps=validation_steps)

In [13]:
model2 = Sequential()

In [14]:
model2.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

model2.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())

model2.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())


model2.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())


model2.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.3))

model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
# model2.add(Dropout(0.3))
model2.add(Dense(1, activation = 'sigmoid'))

In [15]:
model2.compile(optimizer = Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
epochs = 5
steps_per_epoch = train_generator.n // batch_size
validation_steps = test_generator.n // batch_size

history2 = model2.fit(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=test_generator,
                              validation_steps=validation_steps)

Epoch 1/5
652/652 [==============================] - 278s 427ms/step - loss: 0.0117 - accuracy: 0.9950 - val_loss: 3.0895 - val_accuracy: 0.7163
Epoch 2/5
652/652 [==============================] - 300s 460ms/step - loss: 0.0140 - accuracy: 0.9939 - val_loss: 1.2244 - val_accuracy: 0.7756
Epoch 3/5
652/652 [==============================] - 320s 491ms/step - loss: 0.0066 - accuracy: 0.9981 - val_loss: 0.8440 - val_accuracy: 0.8205
Epoch 4/5
652/652 [==============================] - 346s 530ms/step - loss: 0.0069 - accuracy: 0.9973 - val_loss: 0.6044 - val_accuracy: 0.8910
Epoch 5/5
652/652 [==============================] - 317s 486ms/step - loss: 0.0136 - accuracy: 0.9950 - val_loss: 1.3961 - val_accuracy: 0.8109
